# Emulating an actual simulation data from ImQMD
In this tutorial, we will be emulating the simulation data from Improved Quantum Molecular Dynamics (ImQMD) that can be found under the directory `$PROJECTDIR/database/SnSn_e35b6x-1`, where `$PROJECTDIR` is the path of this project repository.

Under `$PROJECTDIR/database/SnSn_e35b6x-1`, the directory structure and naming convention match the default output by ImQMD, which is a fortran executable simulation program. The actual ImQMD software is not included in this repository as it is not part of the intellectual properties of this project. Also, the raw outputs from ImQMD software are not included as their file sizes are huge (many GB for each). Instead, the analyzed results have been extracted using some `C++` program (mainly for speed) and saved as `NP-EK-A16Z6.DAT`, which can be found under every sub-directory in `$PROJECTDIR/database/SnSn_e35b6x-1`.

Since ImQMD simulations can take a long time to run even on HPCC (High Performance Computing Center), our goal is to emulate the analyzed results from simulation inputs (the Skyrme parameters).

Below is the summary:
- Inputs of simulation: `$PROJECTDIR/database/SnSn_e35b6x-1/param.dat`. Each row represents one simulation run.
- Raw outputs of simulation: Not included in this repository.
- Analyzed outputs of simulation: `$PROJECTDIR/database/SnSn_e35b6x-1/$subdir/NP-EK-A16Z6.DAT`. Each sub-directory `$subdir` represents one simulation run. They follow the directory structure of the actual ImQMD software.

Before we can import `skygp`, we need to `sys.path.append()` the directory that contains the relevant source codes. This is done slightly different, depending on whether you are on Google Colab or not. The following script should be able to take care of both cases. Just run it:

In [ ]:
import sys
try:
    import google.colab
    COLAB = True
    print('> You are using Google Colab!')
except:
    COLAB = False

if COLAB:
    print('> Cloning repository to Google Colab...')
    !rm -rf SkyrmeGaussianProcess/
    !git clone https://github.com/Fanurs/SkyrmeGaussianProcess.git
    PROJECT_DIR = '/content/SkyrmeGaussianProcess/'
else:
    PROJECT_DIR = '../'

sys.path.append(PROJECT_DIR)

Then, we should be able to import the module `skygp` as well as other useful libraries.

In [ ]:
%matplotlib inline

# standard modules (e.g. Anaconda)
import numpy as np
import matplotlib.pyplot as plt

# skygp modules
import skygp.gaussian_emulator as gmu
import skygp.data_manager

Troubleshoots (in case):
- If it fails to import any standard modules, make sure you already have them installed.
- If it fails to import any `skygp` modules, make sure you have `sys.path.append()` the correct directory.

# (1) Read in data with `skygp.data_manager`

First, we read in the analyzed results from simulation. This can easily be handled by `skygp.data_manager.SimulationReader` which recognized the directory structure and naming conventions.

In [ ]:
imqmd_reader = skygp.data_manager.SimulationReader('%s/database/SnSn_e35b6x-1/' % PROJECT_DIR)

The physical quantity of interest is something called the "double ratio of neutron-proton yield ratio". The word "double" refers to the fact that we need two collision systems. In this example, we only have:
1. Sn124 + Sn124 at 35 MeV/u
2. Sn112 + Sn112 at 35 MeV/u

The double ratio typically takes the "heavier system" (system 1) divided by the "lighter system" (system 2).

In [ ]:
subdir_fmt_heavy = 'sn124sn124_%03de35b6x-1'
subdir_fmt_light = 'sn112sn112_%03de35b6x-1'

The sub-directory naming convention also tells us other information:
- `%03d` is a placeholder for the code that uniquely identifies which set of simulation parameters was used. The mapping is listed in `$PROJECTDIR/database/SnSn_e35b6x-1/param.dat`.
- `e35` means the beam energy is 35 MeV/u.
- `b6` means the simulation was run with an impact parameter of 6 fm.
- `x-1` means version 1.0 of ImQMD, even though there has not been any newer version yet.

After telling `skygp.data_manager.SimulationReader` where the data is stored, it is fairly straightforward to get the data:

In [ ]:
x_train, y_train = imqmd_reader.get_data(
                        subdir_fmt_heavy, subdir_fmt_light,
                        param_codes=range(50), # we use all available
                        energy_range=[0, 80], # MeV/u
                       )

The outputs are `pandas.DataFrame` objects. We can print them as below:

In [ ]:
display(x_train.head())

In [ ]:
display(y_train.head())

Let's emulate the data.

In [ ]:
emulator = gmu.GaussianEmulator()
emulator.fit(x_train.to_numpy(), y_train.to_numpy())

That's it! We may now inspect the difference between emulation and actual results.

In [ ]:
x_test = np.array([x_train.loc[0]])
y_test = np.array(y_train.loc[0])
y_pred = emulator.predict(x_test)

print('y_test:', y_test.flatten())
print('y_pred:', y_pred.flatten())

fig, ax = plt.subplots(dpi=120, figsize=(4,3), facecolor='white')
ax.plot(y_test.flatten(), label='y_test')
ax.plot(y_pred.flatten(), label='y_pred', linestyle='dashed')
ax.legend()
plt.show()